In [1]:
import duckdb
import pandas as pd

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:default:

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
%%sql
create or replace table competencia_01 as 
select 
    *
from read_csv_auto('/Users/andres/Desktop/master/DM_EyF/datasets/competencia_01_crudo.csv')

,Success


In [3]:
%sql select * from competencia_01 limit 5

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,29183981,202103,1,0,0,50,197,14468.81,125765.29,2389.82,...,0.0,7,0.00,-114954.00,0.00,5938,101050.66,68,0,71811.06
1,29184630,202103,1,0,0,59,322,11901.57,74158.93,18750.68,...,0.0,1,0.00,-40330.15,17.59,4089,26834.09,7,0,3894.36
2,29185433,202103,1,0,0,68,268,847.15,21672.47,481.62,...,0.0,21,4692.00,-1173.00,0.00,7829,1651.36,3,0,1560.09
3,29185587,202103,1,0,0,79,322,4976.94,47735.98,1839.31,...,0.0,1,0.00,-15988.67,0.00,7580,30025.29,11,0,1700.85
4,29185646,202103,1,0,0,60,257,2860.45,37800.71,4035.40,...,0.0,21,380616.14,-97383.25,0.00,7827,359610.70,31,0,15600.90


In [10]:
df = %sql \
    with clientes_marzo as (\
        select numero_de_cliente\
        from competencia_01\
        where foto_mes = '202103'\
    ),\
    clientes_abril as (\
        select numero_de_cliente\
        from competencia_01\
        where foto_mes = '202104'\
    ),\
    clientes_mayo as (\
        select numero_de_cliente\
        from competencia_01\
        where foto_mes = '202105'\
    ),\
    continua as (\
        select numero_de_cliente from clientes_marzo\
        where numero_de_cliente in (select numero_de_cliente from clientes_abril)\
        and numero_de_cliente in(\
select numero_de_cliente from clientes_mayo)\
    ),\
    baja1 as (\
        select numero_de_cliente from clientes_marzo\
        where numero_de_cliente not in(\
            select numero_de_cliente from clientes_abril)\
    ),\
    baja2 as (\
        select numero_de_cliente from clientes_marzo\
        where numero_de_cliente in (select numero_de_cliente from clientes_abril)\
        and numero_de_cliente not in(\
            select numero_de_cliente from clientes_mayo)\
    )\
    select\
    case when numero_de_cliente in (select numero_de_cliente from continua) then 'CONTINUA'\
    when numero_de_cliente in (select numero_de_cliente from baja1) then 'BAJA+1'\
    when numero_de_cliente in (select numero_de_cliente from baja2) then 'BAJA+2' end as clase_ternaria,\
    * from competencia_01\
    where foto_mes in (202103)


In [14]:
df['clase_ternaria']

,clase_ternaria,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,CONTINUA,29183981,202103,1,0,0,50,197,14468.81,125765.29,...,0.0,7.0,0.00,-114954.00,0.00,5938.0,101050.66,68.0,0.0,71811.06
1,CONTINUA,29184630,202103,1,0,0,59,322,11901.57,74158.93,...,0.0,1.0,0.00,-40330.15,17.59,4089.0,26834.09,7.0,0.0,3894.36
2,CONTINUA,29185433,202103,1,0,0,68,268,847.15,21672.47,...,0.0,21.0,4692.00,-1173.00,0.00,7829.0,1651.36,3.0,0.0,1560.09
3,CONTINUA,29185587,202103,1,0,0,79,322,4976.94,47735.98,...,0.0,1.0,0.00,-15988.67,0.00,7580.0,30025.29,11.0,0.0,1700.85
4,CONTINUA,29185646,202103,1,0,0,60,257,2860.45,37800.71,...,0.0,21.0,380616.14,-97383.25,0.00,7827.0,359610.70,31.0,0.0,15600.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163319,CONTINUA,186132954,202103,0,0,0,25,1,0.00,0.00,...,NaN,1.0,0.00,NaN,NaN,15.0,NaN,NaN,NaN,0.00
163320,CONTINUA,186133688,202103,1,0,0,32,1,0.00,0.00,...,NaN,1.0,0.00,NaN,NaN,14.0,NaN,NaN,NaN,0.00
163321,CONTINUA,186133859,202103,0,0,0,63,1,0.00,0.00,...,NaN,1.0,0.00,NaN,NaN,3.0,NaN,NaN,NaN,0.00
163322,CONTINUA,186140768,202103,1,0,0,34,1,0.00,0.00,...,NaN,1.0,0.00,NaN,NaN,9.0,NaN,NaN,NaN,0.00


In [15]:
df['clase_ternaria'].value_counts(dropna=False)

clase_ternaria
CONTINUA    161311
BAJA+1        1050
BAJA+2         963
Name: count, dtype: int64

In [11]:
df.to_csv('../../../datasets/competencia_01.csv',index=False)